In [1]:
#Set up von Spark
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("BigData").config('spark.executor.memory', '8g').config('spark.driver.memory', '8g').getOrCreate()
sc = spark.sparkContext
spark

23/02/03 03:44:49 WARN Utils: Your hostname, mario-hpprobook450g5 resolves to a loopback address: 127.0.1.1; using 141.47.138.65 instead (on interface wlp3s0)
23/02/03 03:44:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 03:44:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from datetime import datetime
from pyspark.sql.functions import col, udf, to_timestamp, year, month
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType, DateType, StringType, FloatType

In [3]:
sc.setJobGroup("job group id", "read all files and print the schema")

#Speicherorte der Daten festlegen 
#Daniel
#folder_Temperature = '/Users/danielwentsch/Desktop/BigData/Data/02_airtemperature/'
#folder_Solar = '/Users/danielwentsch/Desktop/BigData/Data/04_solar/'
#Mario
folder_Temperature = '/home/mario/dwd_data/temperatur_only_10_stations/'
folder_Solar = '/home/mario/dwd_data/solar_only_10_stations/'

#Lufttemperatur
#Daniel
df_Temperatur = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Temperature+'*.txt')
#Sonnenscheindauer
#Daniel
df_Solar = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Solar+'*.txt')

In [4]:
#df_Temperatur.count()

In [5]:
#df_Solar.count()

Nächste Schritte: 
1. Datenstruktur verstehen 
2. Daten bearbeiten und in einen finalen DF schreiben 
-> Zusammenfassen der Daten auf eine Station und einen TimeStamp

In [6]:
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)



Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel) TT_10 (Luftemperatur in 2m Höhe) und TM5_10 (Luftemperatur in 2cm Höhe) und PP_10(Luftdruck)
Ändern der Datentypen von TT_10, TM5_10 und PP_10 in float, MESS_Datum in Timestamp ändern

In [7]:
#Bearbeiten der Struktur in Gewünschtes Format 
#Datentypen bearbeiten 
sc.setJobGroup("job group id", "select only columns that we need, change column type, rename columns")
df_Temperatur = df_Temperatur\
    .withColumn('STATIONS_ID',df_Temperatur.STATIONS_ID.cast(IntegerType()))\
    .withColumn('TM5_10',df_Temperatur.TM5_10.cast(FloatType()))\
    .withColumn('PP_10',df_Temperatur.PP_10.cast(FloatType()))\
    .withColumn('TT_10',df_Temperatur.TT_10.cast(FloatType()))\
    .withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))

#Spalten umbennen
df_Temperatur = df_Temperatur\
    .withColumnRenamed('TT_10','Temperatur_2m')\
    .withColumnRenamed('TM5_10','Temperatur_5cm')\
    .withColumnRenamed('PP_10','Luftdruck')

#Daten droppen
df_Temperatur = df_Temperatur.drop('  QN','RF_10','TD_10','eor')

sc.setJobGroup("job group id", "remove -999 values")
df_Temperatur = df_Temperatur.filter((df_Temperatur.Luftdruck != -999.0))
df_Temperatur = df_Temperatur.filter((df_Temperatur.Temperatur_2m != -999.0))
df_Temperatur = df_Temperatur.filter((df_Temperatur.Temperatur_5cm != -999.0))
df_Temperatur.show(3,False)

+-----------+-------------------+---------+-------------+--------------+
|STATIONS_ID|MESS_DATUM         |Luftdruck|Temperatur_2m|Temperatur_5cm|
+-----------+-------------------+---------+-------------+--------------+
|1766       |2010-01-01 00:00:00|989.7    |-1.3         |-1.8          |
|1766       |2010-01-01 00:10:00|989.7    |-1.3         |-1.8          |
|1766       |2010-01-01 00:20:00|989.8    |-1.3         |-1.8          |
+-----------+-------------------+---------+-------------+--------------+
only showing top 3 rows



In [8]:
df_Temperatur.count()

11252290

In [9]:
df_Solar.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- DS_10: string (nullable = true)
 |-- GS_10: string (nullable = true)
 |-- SD_10: string (nullable = true)
 |-- LS_10: string (nullable = true)
 |-- eor: string (nullable = true)



Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel), SD_10 (10 min-Summe der Sonnenscheindauer)
Ändern der Datentypen von SD_10 in float

In [10]:
#Datentypen bearbeiten
sc.setJobGroup("job group id", "select only columns that we need, change column type, rename columns")
df_Solar = df_Solar\
    .withColumn('SD_10',df_Solar.SD_10.cast(FloatType()))\
    .withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))
#Daten droppen 
df_Solar = df_Solar.drop('  QN','DS_10','GS_10','LS_10','eor')
#Spalte umbennen
df_Solar = df_Solar.withColumnRenamed('SD_10','Sonnenscheindauer')
df_Solar = df_Solar.filter((df_Solar.Sonnenscheindauer != -999.0))
df_Solar.show(3,False)

+-----------+-------------------+-----------------+
|STATIONS_ID|MESS_DATUM         |Sonnenscheindauer|
+-----------+-------------------+-----------------+
|        232|2010-01-01 00:00:00|0.0              |
|        232|2010-01-01 00:10:00|0.0              |
|        232|2010-01-01 00:20:00|0.0              |
+-----------+-------------------+-----------------+
only showing top 3 rows



In [11]:
#df_Solar.count()

Nachdem die beiden Datensätze in die passende Struktur gebracht wurden, können Sie mit einem Join verbunden werden. 
Welchen Join? 
Da für die Auswertung des Spruches, für jede Station und jeden TimeStamp Daten von Solar und Temperature benötigt werden, müssen diese auch von beiden vorhanden sein. 
Daher wird der Inner Join gewählt. 

In [12]:
#Join der DF
sc.setJobGroup("job group id", "join solar and temperature")
df_Final = df_Temperatur.join(df_Solar, (df_Temperatur.STATIONS_ID == df_Solar.STATIONS_ID) & (df_Temperatur.MESS_DATUM == df_Solar.MESS_DATUM),"inner").select(df_Temperatur['*'],df_Solar.Sonnenscheindauer)

In [13]:
df_Final.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- Luftdruck: float (nullable = true)
 |-- Temperatur_2m: float (nullable = true)
 |-- Temperatur_5cm: float (nullable = true)
 |-- Sonnenscheindauer: float (nullable = true)



In [14]:
#df_Final.show(3,False)

In [15]:
#df_Final.count()

Auf der Basis dieser Datentabelle kann nun die Auswertung zur Bauernregel erfolgen. 
Diese Tabelle kann natürlich noch um weitere Wetterdaten erweitert werden, damit andere Bauernregeln oder andere Wettersimulationen durchgeführt werden können.

Bauernregel 1:

"Ist der Januar hell und weiß, wird der Sommer sicher heiß"
1. DF anlegen, welcher alle Kombinationen von Station ID und Jahre besitzt
2. Daten für Januar und Sommer erstellen
3. Auswertung

In [16]:
#DF aus dem df_Final erzeugen welcher alle Kombinationen der StationsIDs, Monate und Jahre enthält
#Neue Spalte Jahr hinzufügen
df = df_Final
df  = df.withColumn("Jahr", year("MESS_DATUM"))
df  = df.withColumn("Monat", month("MESS_DATUM"))
df.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- Luftdruck: float (nullable = true)
 |-- Temperatur_2m: float (nullable = true)
 |-- Temperatur_5cm: float (nullable = true)
 |-- Sonnenscheindauer: float (nullable = true)
 |-- Jahr: integer (nullable = true)
 |-- Monat: integer (nullable = true)



In [17]:
df_Kombinationen =  df.withColumn("ID",concat_ws('','Jahr','STATIONS_ID'))
df_Kombinationen = df_Kombinationen.select("ID",'STATIONS_ID','Jahr')
df_Kombinationen = df_Kombinationen.dropDuplicates(["ID"])
df_Kombinationen.printSchema()

root
 |-- ID: string (nullable = false)
 |-- STATIONS_ID: integer (nullable = true)
 |-- Jahr: integer (nullable = true)



In [18]:
df_Kombinationen.show(3,False)

23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:06:04 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----------+----+
|ID      |STATIONS_ID|Jahr|
+--------+-----------+----+
|20151766|1766       |2015|
|20161766|1766       |2016|
|20183166|3166       |2018|
+--------+-----------+----+
only showing top 3 rows



Pro Jahr sind nur die Temperaturen von 21. Juni bis 23. September und die Sonnenscheindauer im Januar wichtig 

In [20]:
#Zu DF df_Kombinationen eine neue Spalte hinzufügen, welche die Summe an Sonnenstunden pro StationsID und Jahr enthält 
df_Final_Solar_Grouped = df.filter((df.Monat == 12)).groupBy('STATIONS_ID','Jahr','Monat').mean('Sonnenscheindauer')
df_Final_Solar_Grouped = df_Final_Solar_Grouped.withColumnRenamed('avg(Sonnenscheindauer)','Sonnenscheindauer')
df_Kombinationen = df_Kombinationen.join(df_Final_Solar_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Solar_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Solar_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Solar_Grouped.Sonnenscheindauer)

In [22]:
df_Final_Temperatur_Grouped = df.filter((df.Monat == 7)|(df.Monat == 8)).groupBy('STATIONS_ID','Jahr').mean('Temperatur_2m')
df_Final_Temperatur_Grouped = df_Final_Temperatur_Grouped.withColumnRenamed('avg(Temperatur_2m)','Temperatur_2m')
df_Kombinationen = df_Kombinationen.join(df_Final_Temperatur_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Temperatur_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Temperatur_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Temperatur_Grouped.Temperatur_2m)
df_Kombinationen = df_Kombinationen.sort('Jahr')
df_Kombinationen = df_Kombinationen.sort('STATIONS_ID')

In [23]:
df_Kombinationen.printSchema()

root
 |-- ID: string (nullable = false)
 |-- STATIONS_ID: integer (nullable = true)
 |-- Jahr: integer (nullable = true)
 |-- Sonnenscheindauer: double (nullable = true)
 |-- Temperatur_2m: double (nullable = true)



In [24]:
df_Kombinationen.show(3,False)

23/02/03 03:11:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:11:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:11:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:11:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:11:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:11:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:11:59 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+-----------+----+--------------------+------------------+
|ID     |STATIONS_ID|Jahr|Sonnenscheindauer   |Temperatur_2m     |
+-------+-----------+----+--------------------+------------------+
|2015232|232        |2015|0.025358198403370797|20.47743055197714 |
|2012232|232        |2012|0.013385596887166921|18.321281366931494|
|2011232|232        |2011|0.007090725703999453|17.31738350874207 |
+-------+-----------+----+--------------------+------------------+
only showing top 3 rows



In [25]:
#Korrelation zwischen Sonnenschein und Durschsnittstemperatur berechnen
corr_Spruch = df_Kombinationen.stat.corr('Sonnenscheindauer','Temperatur_2m')

23/02/03 03:16:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:16:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:16:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:16:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 03:16:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 03:16:10 WARN RowBasedKeyValueBatch: Calling spill() on

In [26]:
print(corr_Spruch)

0.07188381758199253


Bauernregel 2:

„Wenn's im Dezember nicht wintert, sommert's im Juni auch nicht.“

Hierbei handelt es sich im Prinzip um die entgegengesetzte Regel zu der vorherigen:
Im Juni wird es nicht warm, wenn im Dezember kein richtiger Winter war.
Es stimmt, dass die These zu 65 Prozent eintrifft. Andersrum, wenn es also im Juni nicht „sommert“, wird es im Dezember nicht „wintern“,
stimmt die Bauernregel aber nicht. Warum das so ist, weiß keiner so genau.


In [27]:
# we only need months June and Decemeber
sc.setJobGroup("job group id", "filter only june and december")
df = df_Final
df = df.filter((month("MESS_DATUM") == 6) | (month("MESS_DATUM") == 12))
#df.show()

In [29]:
sc.setJobGroup("job group id", "convert timestamp into month and year")
df = df.select("STATIONS_ID", year("MESS_DATUM").alias("year"),month("MESS_DATUM").alias("month"), "Temperatur_2m")
#df.show()

In [30]:
# calculate average temperature for each year+month+stations_id
sc.setJobGroup("job group id", "calculate average temperature for each year+month+stations_id")
df = df.groupBy("year", "month", "STATIONS_ID").avg("Temperatur_2m")
#df.show()

+----+-----+-----------+--------------------+
|year|month|STATIONS_ID|  avg(Temperatur_2m)|
+----+-----+-----------+--------------------+
|1994|    6|        867|  16.606396293308617|
|2004|   12|        867|-0.42678571584529085|
|2012|    6|       1303|  14.924861110029397|
|1998|   12|       1766|   3.179009858163334|
|1996|   12|       1766| -1.5861970200912785|
|1995|    6|       5440|   14.32541666428248|
|2007|   12|        232|  0.3754928317204732|
|2011|   12|        232|   3.489516128774225|
|1993|   12|       5440|   3.264300160396236|
|2019|    6|        232|   19.16817130015956|
|2020|   12|       5100|   4.743279568033166|
|1996|    6|       5440|  16.966481355155043|
|2014|    6|       3166|  14.149143521211766|
|2014|    6|       1766|  15.994351854589251|
|2015|    6|       5109|  14.535252531249114|
|2011|    6|        891|  15.910046307466649|
|2001|    6|       1303|  14.092196877835606|
|2007|   12|       5100|  1.9235439051199692|
|2015|    6|       5440|  16.60099

In [31]:
sc.setJobGroup("job group id", "new dataframe with only december")
df2 = df.filter(df.month == 12)
#df2.show()

+----+-----+-----------+--------------------+
|year|month|STATIONS_ID|  avg(Temperatur_2m)|
+----+-----+-----------+--------------------+
|2004|   12|        867|-0.42678571584529085|
|1998|   12|       1766|   3.179009858163334|
|1996|   12|       1766| -1.5861970200912785|
|2007|   12|        232|  0.3754928317204732|
|2011|   12|        232|   3.489516128774225|
|1993|   12|       5440|   3.264300160396236|
|2020|   12|       5100|   4.743279568033166|
|2007|   12|       5100|  1.9235439051199692|
|2014|   12|       5440|  2.4165098560359795|
|2019|   12|        232|  2.2346326188788512|
|2009|   12|        891|  1.5916890661769603|
|2017|   12|        891|  4.5894041186996395|
|1995|   12|       1766|  -1.556287943330346|
|2013|   12|       1766|   5.322535843062665|
|1999|   12|       3166| -0.6046184778798642|
|1995|   12|        867|  -1.564560928765858|
|2004|   12|        891|   3.748443536922569|
|2012|   12|       1303|  4.3271462884363086|
|2020|   12|       1766|   5.14173

In [32]:
from pyspark.sql.functions import lit

sc.setJobGroup("job group id", "adding additional month and year columns")
df2 = df2.withColumnRenamed('avg(Temperatur_2m)', 'temperature_december')
df2 = df2.withColumnRenamed('year', 'year_december')
df2 = df2.withColumnRenamed('month', 'month_december')
df2 = df2.withColumn('year_june', df2.year_december + 1)
df2 = df2.withColumn('month_june', lit(6))
#df2.show()

+-------------+--------------+-----------+--------------------+---------+----------+
|year_december|month_december|STATIONS_ID|temperature_december|year_june|month_june|
+-------------+--------------+-----------+--------------------+---------+----------+
|         2004|            12|        867|-0.42678571584529085|     2005|         6|
|         1998|            12|       1766|   3.179009858163334|     1999|         6|
|         1996|            12|       1766| -1.5861970200912785|     1997|         6|
|         2007|            12|        232|  0.3754928317204732|     2008|         6|
|         2011|            12|        232|   3.489516128774225|     2012|         6|
|         1993|            12|       5440|   3.264300160396236|     1994|         6|
|         2020|            12|       5100|   4.743279568033166|     2021|         6|
|         2007|            12|       5100|  1.9235439051199692|     2008|         6|
|         2014|            12|       5440|  2.4165098560359795|  

In [34]:
sc.setJobGroup("job group id", "join two dataframes")
df2 = df2.join(df, (df2.year_june == df.year) & (df2.month_june == df.month) & (df2.STATIONS_ID == df.STATIONS_ID)).select(df2["*"], df["avg(Temperatur_2m)"])
df2 = df2.withColumnRenamed('avg(Temperatur_2m)', 'temperature_june')
#df2.show()

+-------------+--------------+-----------+--------------------+---------+----------+------------------+
|year_december|month_december|STATIONS_ID|temperature_december|year_june|month_june|  temperature_june|
+-------------+--------------+-----------+--------------------+---------+----------+------------------+
|         1993|            12|        867|   2.357974910526119|     1994|         6|16.606396293308617|
|         2011|            12|       1303|   5.188978491840036|     2012|         6|14.924861110029397|
|         1994|            12|       5440|   3.384896769919023|     1995|         6| 14.32541666428248|
|         2018|            12|        232|  2.9756272414058285|     2019|         6| 19.16817130015956|
|         1995|            12|       5440| -0.9267697132402851|     1996|         6|16.966481355155043|
|         2013|            12|       3166|   2.742562724210616|     2014|         6|14.149143521211766|
|         2013|            12|       1766|   5.322535843062665| 

In [35]:
sc.setJobGroup("job group id", "calculate correlation")
df2.corr("temperature_december", "temperature_june", method="pearson")

0.2159190300045076

In [ ]:
#df2.coalesce(1).write.mode("overwrite").option("header", "true").csv("/hdfs://192.168.199.80:9000/user/mario/output_files/10minutenwerte_only_100_result")